In [424]:
import io
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold
%matplotlib inline
from scipy.spatial.distance import cdist

# 2.a

### Load Multiple Feature Dataset

In [145]:
def read_file(filename):
    r = io.open(filename, encoding='utf8').readlines()
    X = []
    Y = []
    for i in r:
        if i.split()[0] != '#':
            tmp = i.split()
            X.append([float(tmp[o]) for o in range(len(tmp)-1)])
            Y.append([float(tmp[-1])])
    return X,Y

### Function to Map  to Higher Dimension

In [146]:
def map_high_dimension(Data,degree):
    polyfeat_object = PolynomialFeatures(degree)
    hd_data = polyfeat_object.fit_transform(Data)
    return hd_data

In [147]:
Z,Y = read_file("mvar-set1.dat.txt")
degrees = [2,3,4,5,6]
Z_hd_dictionary = {}
for i in degrees:
    Z_hd_dictionary[i] = map_high_dimension(Z,i)
print "Data Matrix mapped to Higher dimemsion of degree 3"
print Z_hd_dictionary[3]

Data Matrix mapped to Higher dimemsion of degree 3
[[  1.00000000e+00   1.67346939e+00   4.48979592e-01 ...,   1.25736725e+00
    3.37342434e-01   9.05065066e-02]
 [  1.00000000e+00  -4.08163265e-02   5.30612245e-01 ...,   8.83985414e-04
   -1.14918104e-02   1.49393535e-01]
 [  1.00000000e+00  -7.75510204e-01  -1.59183673e+00 ...,  -9.57356204e-01
   -1.96509958e+00  -4.03362545e+00]
 ..., 
 [  1.00000000e+00  -2.00000000e+00  -6.93877551e-01 ...,  -2.77551020e+00
   -9.62932112e-01  -3.34078488e-01]
 [  1.00000000e+00  -1.83673469e+00   1.02040816e+00 ...,   3.44244320e+00
   -1.91246844e+00   1.06248247e+00]
 [  1.00000000e+00  -2.85714286e-01   1.42857143e+00 ...,   1.16618076e-01
   -5.83090379e-01   2.91545190e+00]]


# 2.b


### Find Theta - Linear Regression

In [148]:
def find_theta(Data,Labels):
    return np.dot(np.linalg.pinv(Data),np.array(Labels))

### Predict 

In [149]:
def predict(t,z):
    predictions = []
    for i in z:
        predictions.append(np.dot(t.transpose(),np.array(i)))
    return predictions

### Mean Square Error

In [150]:
def mean_square_error(pred,y):
    return sum([(i-j)**2 for i,j in zip(pred,y)])/len(pred)

### Finding Theta for all High Dimension Feature Space

In [151]:
Z_hd_theta = {}
for i in Z_hd_dictionary:
    Z_hd_theta[i] = find_theta(Z_hd_dictionary[i],Y)
print "Theta value of high dimensional data set of degree 3"
print Z_hd_theta[3]

Theta value of high dimensional data set of degree 3
[[  1.02230218e+00]
 [  1.00248539e+00]
 [  1.00537442e+00]
 [ -1.26097513e-02]
 [ -8.47393334e-03]
 [ -6.43213699e-03]
 [ -2.31159943e-03]
 [ -1.64092663e-02]
 [  5.84897669e-04]
 [  3.15715316e-03]]


### Predict Using the Theta 

In [152]:
Y_hd_predict = {}
for i in Z_hd_dictionary:
    Y_hd_predict[i] = predict(Z_hd_theta[i],Z_hd_dictionary[i])
print "Predicted and actual values of high dimensional data set of degree 3(first 25 values)"
print "Predcited\t\tActual"
for i in range(25):
    print Y_hd_predict[3][i],"\t\t",Y[i]  

Predicted and actual values of high dimensional data set of degree 3(first 25 values)
Predcited		Actual
[ 3.07736342] 		[2.861980967575097]
[ 1.51365053] 		[0.5195252261416367]
[-1.38696723] 		[-2.245760962347731]
[-2.32471805] 		[-1.561989674513609]
[ 1.34557589] 		[1.887592936186108]
[ 1.22770058] 		[0.9884311316371547]
[-1.44192419] 		[-1.085317769937612]
[ 1.29287274] 		[0.8038785273144268]
[ 2.49595503] 		[3.18204127016383]
[ 2.53802138] 		[2.351083268524211]
[ 3.51292129] 		[3.626071229401944]
[-0.17188473] 		[0.4634078745339167]
[ 0.85542858] 		[1.487055672161672]
[ 2.3113871] 		[2.142605152020115]
[-0.46229492] 		[-1.110575348573327]
[ 3.10202926] 		[3.929068438319104]
[-1.94229288] 		[-1.927879404879771]
[-0.56049346] 		[-0.1816051916186905]
[ 1.90961133] 		[1.71526336810711]
[ 0.62064496] 		[0.7650547285963669]
[-0.87883622] 		[-1.280690811878441]
[ 3.69919814] 		[3.876430712322037]
[ 0.59320767] 		[0.5123972650771516]
[ 0.42276492] 		[0.3184214808888129]
[ 2.76858808] 		[2.4

### Finding Mean Square Errors

In [153]:
MSE_mean ={}
for i in Y_hd_predict:
    MSE_mean[i] = mean_square_error(Y_hd_predict[i],Y)
print "Mean Square Errors of High Dimensional feature space"    
print"Degree\tMSE"
for i in MSE_mean:
    print i,"\t",MSE_mean[i][0]
    

Mean Square Errors of High Dimensional feature space
Degree	MSE
2 	0.258256248391
3 	0.257655351948
4 	0.256493824893
5 	0.256237649361
6 	0.255132612013


### Cross Validation for n folds using manualy created fit and predict functions; Finding the Training Error and Testing Error

In [154]:
def fold_indices(size, folds):
    block_list = [size//folds for i in range(folds)]
    if size%folds != 0:
        for i in range(n%n_folds):
            block_list[i] += 1
    current = 0
    indices = []
    for x in block_list:
        m,n = current, current + x
        indices.append((m,n))
        current = n
    return indices

In [155]:
def cross_validation(data,labels,folds):
    k_index = fold_indices(len(labels), folds)
    Theta_list = []
    training_MSE_list =[]
    testing_MSE_list = []
    for i in k_index:
        Z_test = data[i[0]:i[1]]
        Y_test = labels[i[0]:i[1]]
        Z_train = np.ndarray([])
        Y_train = np.ndarray([])
        for j in k_index:
            if i != j:
                Z_train = Z_train + data[j[0]:j[1]]
                Y_train = Y_train + labels[j[0]:j[1]]
        Theta  = find_theta(Z_train,Y_train)
        training_MSE = mean_square_error(predict(Theta,Z_train), Y_train)
        pred = predict(Theta,Z_test)
        MSE = mean_square_error(pred, Y_test)
        Theta_list.append(Theta)
        testing_MSE_list.append(MSE)
        training_MSE_list.append(training_MSE)
    return Theta_list, testing_MSE_list, training_MSE_list

In [156]:
thetas,test_mse,training_mse = cross_validation(Z_hd_dictionary[6],Y,10)
print("The Training Errot and Testing Error for Linear Regression on High Dimensional Feature Space of degree 6 for 10 folds")
print("Training Error\tTesting Error")
for i in range(len(training_mse)):
    print "%f\t%f" %(test_mse[i][0],training_mse[i][0])

The Training Errot and Testing Error for Linear Regression on High Dimensional Feature Space of degree 6 for 10 folds
Training Error	Testing Error
1.721218	2.025014
5.277094	2.139538
6.931781	2.052602
7.224948	2.054735
8.520127	2.129154
7.227964	1.953868
7.575019	2.151559
7.194835	2.074939
8.557931	2.150347
7.581050	2.147773


# 2.c

### Iterative Solution

In [157]:
def gradient_descent(z, y, theta_assume, learning_rate, interation_count):
    theta = np.ones(len(z[0]))
    theta.fill(theta_assume)
    for i in range(interation_count):
        pred = predict(theta,z)
        sumtheta = np.zeros(len(z[0]))
        for i in range(len(pred)):
            sumtheta = sumtheta + ((pred[i] - y[i])*z[i])
        theta = theta - learning_rate*sumtheta
    return theta

In [158]:
xy =gradient_descent(Z_hd_dictionary[3][0:2000], Y[0:2000], 0.1, 0.00005, 600)

In [159]:
xy

array([ 1.01512903,  1.00395836,  0.98701184, -0.01487465, -0.01156393,
       -0.00260941, -0.00413616, -0.01150208,  0.00471191,  0.00403633])

In [160]:
train_data,test_data = Z_hd_dictionary[3][0:2000],Z_hd_dictionary[3][2000:]
train_y,test_y = Y[0:2000],Y[2000:]

In [161]:
def iterative_solution(z, y, theta_assume, learning_rate, interation_count,z_test):
    theta_grad =gradient_descent(z, y, theta_assume, learning_rate, interation_count)
    print theta_grad
    pred_is = predict(np.array(theta_grad),z_test)
    return pred_is

In [162]:
def explicit_solution(z, y,z_test):
    theta_es = find_theta(z,y)
    print theta_es
    pred_es = predict(theta_es,z_test)
    return pred_es

In [163]:
iterative_solution(train_data, train_y, 0.1, 0.00005, 600,test_data)

[ 1.01512903  1.00395836  0.98701184 -0.01487465 -0.01156393 -0.00260941
 -0.00413616 -0.01150208  0.00471191  0.00403633]


[1.2453408053522952,
 1.1140300544616561,
 -2.0256944149678606,
 1.1825210696628787,
 1.4754847039209054,
 -2.1153502532509045,
 1.3052016163516986,
 1.0352919283873321,
 3.5184167498795911,
 0.92894920536244963,
 0.22070448774998377,
 2.223974873624929,
 -1.7970791341422869,
 2.8363442388821634,
 -0.58256830181926789,
 -0.28988428190842863,
 2.9855461158733396,
 -0.37337414156988752,
 -2.7702348477503196,
 2.4529754146273972,
 1.347953967461883,
 0.28624968163769982,
 0.83056805479630702,
 3.663961493802558,
 0.13060148090415388,
 2.3042645280978515,
 -0.088237048261898043,
 3.3646871534739082,
 1.4092915456051516,
 -0.54152881081404447,
 -0.48883290767341009,
 4.4251144759797496,
 -1.7865229914566298,
 2.2246528174791886,
 0.12177889890497418,
 3.3140330597593146,
 -2.5369110826153802,
 -0.61779266030688085,
 0.069726638729551191,
 1.9024477634253296,
 2.3857084743103907,
 0.10832136002332501,
 1.8210083504857824,
 0.35542558624348308,
 4.7601559129812063,
 -1.126943160735796,
 0.128

In [164]:
explicit_solution(train_data, train_y,test_data)

[[ 1.01512825]
 [ 1.0039924 ]
 [ 0.98704724]
 [-0.01487432]
 [-0.01156381]
 [-0.00260912]
 [-0.0041459 ]
 [-0.01150636]
 [ 0.00470772]
 [ 0.00402614]]


[array([ 1.24534673]),
 array([ 1.11402977]),
 array([-2.02569626]),
 array([ 1.18252498]),
 array([ 1.47548655]),
 array([-2.11535095]),
 array([ 1.30520167]),
 array([ 1.03529078]),
 array([ 3.5184259]),
 array([ 0.92894607]),
 array([ 0.2207029]),
 array([ 2.22397577]),
 array([-1.79708893]),
 array([ 2.83637781]),
 array([-0.58255668]),
 array([-0.28991568]),
 array([ 2.98557441]),
 array([-0.37340754]),
 array([-2.77017439]),
 array([ 2.45301157]),
 array([ 1.3479599]),
 array([ 0.28622658]),
 array([ 0.83058148]),
 array([ 3.66397428]),
 array([ 0.13058729]),
 array([ 2.30430027]),
 array([-0.08823855]),
 array([ 3.36470811]),
 array([ 1.40930116]),
 array([-0.54156879]),
 array([-0.48883915]),
 array([ 4.42507302]),
 array([-1.78654268]),
 array([ 2.22468664]),
 array([ 0.12175181]),
 array([ 3.31406803]),
 array([-2.53687717]),
 array([-0.61778262]),
 array([ 0.06975406]),
 array([ 1.902463]),
 array([ 2.38568653]),
 array([ 0.10829498]),
 array([ 1.82103043]),
 array([ 0.35540

# 2.d

### Gaussian Kernal Function

In [322]:
def kernel_gram_matrix(x,y,sigma):
    d = cdist(x,y,'sqeuclidean')    
    return np.exp((-1/2)*(d/sigma**2))

In [323]:
def kernel_weights(gram,labels):
    return np.linalg.solve(gram,labels)

In [337]:
def gaussian_predict(alpha,x,x_sample,sigma):
    kernel_sample = kernel_gram_matrix(x,x_sample,sigma)
    return np.dot(alpha.transpose(),kernel_sample)

In [328]:
def gaussian_fit(x,y,sigma):
    G = kernel_gram_matrix(x,x,sigma)
    alpha = kernel_weights(G,y)
    return alpha

In [421]:
def gaussian_theta(alpha,x):
    return np.dot(alpha.transpose(),x)

In [418]:
fjh = gaussian_fit(Z_hd_dictionary[2],Y,0.11111)

In [419]:
predd = gaussian_predict(fjh,Z_hd_dictionary[2],Z_hd_dictionary[2],0.11111)

In [422]:
gaussian_theta(fjh,Z_hd_dictionary[2])

array([[ 1905.89401906,  3107.0743663 ,  3082.7617422 ,  3049.15325924,
          -33.92809003,  3074.93978972]])

In [423]:
zz = 0
for i in range(len(predd.transpose())):
    cv = (predd.transpose()[i] - Y[i])**2
    zz = zz + cv
print zz/len(predd)

[  4.90950493e-27]
